<a href="https://colab.research.google.com/github/tchih11/qiita_eda/blob/main/notebooks/make_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install neologdn
!pip install category_encoders

import glob
import os
import pathlib
import tarfile

import category_encoders as ce
import neologdn
import pandas as pd

In [2]:
# Livedoorニュースのファイルをダウンロード
! wget "https://www.rondhuit.com/download/ldcc-20140209.tar.gz"

# rawデータ解凍
tar = tarfile.open("./ldcc-20140209.tar.gz", "r:gz")
tar.extractall(".")
tar.close()

--2021-02-08 03:41:54--  https://www.rondhuit.com/download/ldcc-20140209.tar.gz
Resolving www.rondhuit.com (www.rondhuit.com)... 59.106.19.174
Connecting to www.rondhuit.com (www.rondhuit.com)|59.106.19.174|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8855190 (8.4M) [application/x-gzip]
Saving to: ‘ldcc-20140209.tar.gz’

ldcc-20140209.tar.g 100%[===================>]   8.44M  1.71MB/s    in 4.9s    

2021-02-08 03:41:59 (1.71 MB/s) - ‘ldcc-20140209.tar.gz’ saved [8855190/8855190]



In [3]:
df = pd.DataFrame(columns=["text", "category"])

for file_path in pathlib.Path("./text").glob("**/*.txt"):
    f_path = pathlib.Path(file_path)
    file_name = f_path.name
    category_name = f_path.parent.name

    # 特殊ファイルはスキップ
    if file_name in ["CHANGES.txt", "README.txt", "LICENSE.txt"]:
        continue

    with open(file_path, "r") as f:
        text_all = f.read()
        text_lines = text_all.split("\n")
        url, time, title, *text = text_lines

        # text前処理
        text = "".join(text)
        text = text.strip()
        text = neologdn.normalize(text)
        text = text.replace(' ', '')
        text = text.replace('　', '')

        df.loc[file_name] = [text,category_name]
        
# インデックスに使用していたファイル名を削除
df.reset_index(inplace=True,drop=True)

# カテゴリをエンコーディング
ce_oe = ce.OrdinalEncoder(cols='category',handle_unknown='impute',return_df=True)
df = ce_oe.fit_transform(df)
# モデリング時に、ラベルは0から始まらないとエラーになる
df["category"] = df.category - 1

# 重複する記事を削除
df = df.drop_duplicates()
df.reset_index(inplace=True,drop=True)

In [4]:
# 保存
if not os.path.exists("./data"):
    os.mkdir("./data")

# 全体の2割の文章数
df = df.sample(frac=1, random_state=123).reset_index(drop=True)
len_0_2 = len(df) // 5

# 前から2割をテストデータとする
test_df = df[:len_0_2].reset_index(drop=True)
test_df.to_pickle("./data/test_df.pkl")
print(test_df.shape)

# 前2割からを訓練&検証データとする
train_eval_df = df[len_0_2:].reset_index(drop=True)
train_eval_df.to_pickle("./data/train_eval_df.pkl")
print(train_eval_df.shape)

(1471, 2)
(5888, 2)


In [5]:
train_eval_df

,text,category
0,誰も教えてくれなかったファッションのヒミツを、内面の魅力から解き明かし、あなたが着るべきスー...,4
1,ARROWSZISW11Fのケータイアップデートが再開!KDDIおよび沖縄セルラーは23日、...,8
2,11日、映画『ヒミズ』のヒットを願って、市谷亀岡八幡宮にて祈願イベントが行われた。ヴェネチア...,1
3,「あたしゃねぇ、もう30年も銭湯やってるけど、自分の都合で店しめたことなんか、ないんだよ!休...,2
4,オフィスで会議用などの大量の資料組をする際、複合機で印刷と同時に穴あけやホチキス止めも行って...,6
...,...,...
5883,先日冬の訪れを告げる木枯らし1号が東京都心で観測された。気温が急に下がったり上がったりで、最...,6
5884,ウェディングドレスのデザイナーは「アレキサンダー・マックイーン」のサラ・バートンついに世界中...,5
5885,本日27日(日本時間)に開催された「第84回アカデミー賞」授賞式にて、3月1日(木・映画の日...,1
5886,友達とシェアしたい写真をFacebookにアップしたら、相手にも気づいてほしい!1人や2人な...,3
